In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core import Document
from llama_index.core import download_loader, SimpleDirectoryReader, VectorStoreIndex
from llama_index.readers.file import CSVReader, PandasCSVReader
import os
import openai
import pandas as pd

In [11]:
from llama_index.core import Settings

folder_path = './'

with open(f'{folder_path}OpenAI_API_Key.txt', 'r') as f:
    openai.api_key = f.read().strip()

llm = OpenAI(
    model="gpt-4o-mini"
)

Settings.llm = llm

In [12]:
fasion_df = pd.read_csv(f'{folder_path}dataset/myntra_dataset.csv')

In [30]:

documents = [
    Document(
        text=row['name'],
        metadata={
            "id": row['p_id'],
            "description": row['description'],
            "img": row['img'],
            "price": row['price'],
            "color": row['colour'],
            "brand": row['brand']
        },
    )
    for _, row in fasion_df.iterrows()
]

In [31]:
from llama_index.core import StorageContext, load_index_from_storage
database_path = f"{folder_path}/database"
store_index = True
try:
    print("loading index from the local storage")
    storage_context = StorageContext.from_defaults(persist_dir=database_path)
    index = load_index_from_storage(storage_context=storage_context, embed_model=Settings.embed_model)
except:
    print("Creating index from documents")
    index =  VectorStoreIndex.from_documents(documents)
    if store_index:
        index.storage_context.persist(persist_dir=database_path)


loading index from the local storage
Creating index from documents


In [32]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

#### Using Retriever Engine

In [33]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(
    retriever = retriever,
)
query_engine

In [34]:
query_engine_response = query_engine.query("Stylish outfit for winter season.")

print(query_engine_response)

For a stylish winter outfit, consider the following options:

1. **KASSUALLY Women Beautiful Off-White Solid Cardigan Set** - This elegant sweater features a cosy V-neck and long sleeves, perfect for layering. Priced at 2999, it offers a youthful and trendy look.

2. **iki chic Women Stylish Black Washed Jacket** - A fashionable choice with a stand collar and sleeveless design, this jacket is ideal for a chic winter ensemble. It is available for 4599.

3. **FREAKINS Women Lovely Olive Solid Jacket** - This classy jacket comes with a solid pattern and long sleeves, providing warmth and style. It is priced at 2399.

4. **FREAKINS Women White Printed Jacket** - Featuring a unique printed pattern and spread collar, this jacket stands out beautifully. It is also available for 2399.

5. **DressBerry Women Stunning Blue Solid Sweater** - This sweater, with its enchanting V-neck and long sleeves, is a great option for staying warm while looking stylish. It is priced at 2299.

These pieces can 

### Observation:
 1. Metadata not included in the response.

### Using default Query Engine

In [35]:
default_engine = index.as_query_engine()

default_engine_response = default_engine.query("Stylish outfit for winter season.")

print(default_engine_response)

Consider a cozy off-white cardigan set from KASSUALLY, featuring a V-neck and long sleeves, perfect for layering during the winter. Alternatively, a trendy black sleeveless jacket from iki chic with a stand collar can add a fashionable touch to your winter wardrobe. Both options are stylish and suitable for keeping warm.


### Observation
 1. Brief output.

### Using default chat engine from index.

In [17]:
chat_engine = index.as_chat_engine()

In [18]:
chat_response = chat_engine.query("stylish sleeveless tops")

print(chat_response)

A stylish option for a sleeveless top is the classic white solid bra top and vest from abof. It features a square neck and is made from a comfortable cotton blend, making it perfect for a fashionable impression at any get-together.


In [19]:
print(chat_response.response)

A stylish option for a sleeveless top is the classic white solid bra top and vest from abof. It features a square neck and is made from a comfortable cotton blend, making it perfect for a fashionable impression at any get-together.


In [20]:
for node in chat_response.source_nodes:
    print(node.metadata['description'], end="\n")
    print(node.metadata['img'], end="\n\n")

Steal the spotlight when you wear this trendy and versatile top for a get-together with pals. Featuring an attractive solid pattern and V-neck,this top is sure to emphasize your overall appeal.,Black colour,Classic solid pattern,V-neck,Three-quarter puff sleeves,Polyester,machine wash,,Trend Alert,Puff sleeves are typically gathered at the shoulder and puffed in between. They then gather back at the end creating a balloon-like look. Comfortable and stylish,they have been in vogue since the 14th century.Polyester,Machine washThe model (height 58) is wearing a size S
http://assets.myntassets.com/assets/images/17012370/2022/2/17/cbc6ef85-2c35-4c13-9e24-c32595435b721645096897132-Sera-Women-Tops-4831645096896516-1.jpg

Pick this fashionable top and make a worthy impression at your next get-together. The solid pattern and square neck of this beautiful top emphasize its overall appeal.,Classic white colour,Solid pattern,Square neck,Sleeveless,Cotton blend,machine wash,Trend Alert,Bra tops and

### Using Condense Question chat engine.

In [21]:
from llama_index.core.chat_engine import ContextChatEngine,CondenseQuestionChatEngine

context_chat_engine = CondenseQuestionChatEngine.from_defaults(
    retriever=retriever,
    llm=llm,
    query_engine=query_engine
)

In [22]:
context_chat_response = context_chat_engine.chat("stylish sarees in red and pink color")

print(context_chat_response)

Here are some stylish sarees in red and pink color:

1. **Satrani Women Red & Pink Geometric Printed Saree**
   - Description: Geometric printed saree with a printed border.
   - Fabric: 100% Chiffon.
   - Length: 5.5 metres plus 0.8 metre blouse piece.
   - Price: 4974.

2. **Fashionuma Pink & Red Ethnic Motifs Pure Silk Saree**
   - Description: Ethnic motifs printed saree with a printed border.
   - Fabric: Pure Silk.
   - Length: 5.5 metres plus 0.8 metre blouse piece.
   - Price: 2999.

These options combine elegance and style, perfect for various occasions.


#### Observations:
 1. The response does not have metadata retreived along with the search result.

In [23]:
chat_history= [ChatMessage(role="user", content="stylish sarees in red and pink color")]

#### Trying to continue conversation using chat history.

In [24]:

context_chat_response = context_chat_engine.chat("Can you suggest some more options please?", chat_history)

print(context_chat_response)

Here are some stylish saree options in red and pink colors:

1. **Fashionuma Pink & Red Ethnic Motifs Pure Silk Saree**
   - Price: ₹2999
   - Description: Features ethnic motifs printed on pure silk with a printed border. Comes with an unstitched blouse piece.

2. **Sangria Pink & Red Floral Organza Saree**
   - Price: ₹3399
   - Description: A floral printed saree made of organza, with no border. Includes an unstitched blouse piece.

These options showcase a blend of traditional and contemporary designs, perfect for various occasions.


### Observation:
 1. Even shorter response which does not seem to be retrieved from the index.